This notebook is used to create the meta-dataset for all reczilla meta-learner experiments.

### v1.1: 

This version uses the new experiments, with experiment names starting with `neurips-`.

New in v1.1: includes almost all experiments. This only leaves out some knn experiments, which are still running.

In [1]:
# define the results file here. this file should be created by the script process_inbox.py
results_csv = "/Users/zzzzzz/research/active_projects/reczilla/results/results.csv"

# name of the resulting meta-dataset
meta_dataset_name = "metadata-v1.1"

# define a function that takes a row as input an returns True if the row should be included in the meta-dataset and false otherwise
def include_row(row):
    if row["experiment_name"].startswith("neurips-"):
        return True
    else:
        return False


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(results_csv, sep=";")

/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,13,14,16,17,31,32,38,40,41,42,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Filter the results based on the function "include_row"

In [3]:
### keep experiments indicated by function include_row
experiment_prefix = "full-experiment-"
keep_rows = df.apply(include_row, axis=1)
df_expt = df.loc[keep_rows, :]

### if there are any rows with no metrics, drop them...
metric_col_list = [c for c in df_expt.columns if c.startswith("test_metric_") or c.startswith("val_metric_")]
df_expt.loc[:, "all_na_metrics"] = df_expt[metric_col_list].isna().all(axis=1)
df_expt = df_expt.loc[~df_expt["all_na_metrics"], :]

/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### sanity check 1: make sure that for each alg_name + hyperparameters_source combination all hyperparams are the same. 

**Note:** if any warnings are printed here, we need to investigate. 

In [4]:
param_list = [c for c in df_expt.columns if c.startswith("param_")]

for param_name in param_list:
    x = df_expt.groupby(["alg_name", "hyperparameters_source"])[param_name].agg(
        {"one_unique_param": lambda x: len(x.unique()) == 1}
    ).reset_index()
    if not all(x["one_unique_param"]):
        print(f"WARNING: for param {param_name}, params aren't uniquely defined:")
        print(x[~x["one_unique_param"]])

# use a line like this to diagnose issues:
# df_expt[(df_expt["alg_name"] == "ItemKNNCF_asymmetric") & (df_expt["hyperparameters_source"] == "random_0")][["experiment_name"] + param_list]

/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


### final cleanup:
- define columns "alg_family" (e.g., UserKNN) and "parameterized_alg_name" (e.g., UserKNN_hparams_1). 
- param cols
- metric cols
- runtime
- dataset name
- dataset split name
- dataset source from gcloud

In [5]:
keep_cols = [
    "alg_name",
    "dataset_name",
    "split_name",
    "original_split_path",
    "hyperparameters_source", 
    "time_on_val",
    "time_on_test",
    "time_on_train",
] + metric_col_list + param_list

final_df = df_expt.loc[:, keep_cols]

final_df.rename(columns={"alg_name": "alg_family"}, inplace=True)

# assign a unique identifier for each alg+hyperparam pair
final_df.loc[:, "alg_param_name"] = ""

sep = ":"

# for all algs except for KNN, this is straightforward:
knn_rows = final_df["alg_family"].str.contains("KNN")
final_df.loc[~knn_rows, "alg_param_name"] = final_df.loc[~knn_rows, :].apply(lambda x: x["alg_family"] + sep + x["hyperparameters_source"], axis=1)

# for KNN rows, the alg family is UserKNN or ItemKNN, not including similarity...
knn_basename = final_df.loc[knn_rows, "alg_family"].apply(lambda x: x.split("_")[0])
knn_sim = final_df.loc[knn_rows, "alg_family"].apply(lambda x: x.split("_")[1])
final_df.loc[knn_rows, "alg_family"] = knn_basename  # either UserKNN or ItemKNN
final_df.loc[knn_rows, "alg_param_name"] = knn_basename + sep + knn_sim + "_" + final_df.loc[knn_rows, "hyperparameters_source"]


# final_df.loc[:, "alg_param_name"] = final_df.apply(lambda x: x["alg_family"] + ":" + x["hyperparameters_source"], axis=1)

### sanity check 2: check whether there are multiple rows for each alg + hyperparam + dataset combination. if so, make sure the results are identical

In [8]:
z = final_df.groupby(["alg_param_name", "original_split_path"]).size().rename("num_rows").reset_index()
z_multiple = z[z['num_rows'] > 1]
print(f"number of alg+param+dataset combinations with multiple results: {len(z_multiple)}")

number of alg+param+dataset combinations with multiple results: 0


In [7]:
if len(z_multiple) > 0:

    # for each combination with multiple rows, find the difference in metric values between first two rows
    for i, (a_p_n, o_s_p) in z_multiple[["alg_param_name", "original_split_path"]].iterrows():
        row_data = final_df.loc[(final_df["alg_param_name"] == a_p_n) & (final_df["original_split_path"] == o_s_p), metric_col_list].values
        if np.linalg.norm(row_data[0, :] - row_data[1, :]) > 1e-5:
            print(f"WARNING: difference between multiple results is non-negligible for alg={a_p_n}, dataset={o_s_p}")
    
    # drop duplicates, keep first row only.
    print("dropping duplicates...")
    final_df.drop_duplicates(subset=["alg_param_name", "original_split_path"], keep="first", inplace=True)


dropping duplicates...


## write results

In [9]:
# write the dataset to file
out_file = f"./meta_datasets/{meta_dataset_name}.pkl"

final_df.to_pickle(out_file)

# to read this file, do:
# final_df_2 = pd.read_pickle(out_file)

### Basic numbers

In [10]:
print(f"total number of parameterized algs: {len(final_df['alg_param_name'].unique())}")
print(f"total number of datasets: {len(final_df['original_split_path'].unique())}")
print(f"total number of rows: {len(final_df)}")
print('--------------------------------------')
print(f"total number of rows for each alg family: {final_df.groupby('alg_family').size()}")
print('--------------------------------------')
print(f"total number of rows for each dataset: {final_df.groupby('original_split_path').size()}")

total number of parameterized algs: 1476
total number of datasets: 85
total number of rows: 78929
--------------------------------------
total number of rows for each alg family: alg_family
CoClustering                           5106
EASE_R_Recommender                     4376
GlobalEffects                            85
IALSRecommender                        3502
ItemKNNCF                             12847
MatrixFactorization_AsySVD_Cython      4254
MatrixFactorization_BPR_Cython         5659
MatrixFactorization_FunkSVD_Cython     4938
NMFRecommender                         2957
P3alphaRecommender                     5816
PureSVDRecommender                     6132
RP3betaRecommender                     5900
Random                                   85
SLIMElasticNetRecommender              4706
SLIM_BPR_Cython                        5176
SlopeOne                                 48
TopPop                                   85
UserKNNCF                              7257
dtype: int64
-----